<a href="https://colab.research.google.com/github/AShipulin/usurt/blob/main/%D0%9B%D0%9F%D0%A2%D0%A1_%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BB%D0%B0%D0%BD_cplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install cplex docplex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.9/645.9 kB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 13.6 MB/s eta 0:00:00
  Created wheel for docplex: filename=docplex-2.29.245-py3-none-any.whl size=685439 sha256=08722ce3e5d3c8608c500a89fd6d38d4854fdc50f5adee7adaf438ed39cd4589
  Stored in directory: /root/.cache/pip/wheels/be/37/0a/5f5f9ed14d637a664ad676493d55654f655a099c7016808739
Successfully built docplex


In [ ]:
from docplex.mp.model import Model

# Создаем модель
model = Model(name='Transportation_Problem')

# Данные задачи
suppliers = ['a1', 'a2']
supply = {'a1': 10, 'a2': 20}

consumers = ['b1', 'b2', 'b3']
demand = {'b1': 8, 'b2': 13, 'b3': 9}

costs = {
    ('a1', 'b1'): 7,
    ('a1', 'b2'): 9,
    ('a1', 'b3'): 21,
    ('a2', 'b1'): 20,
    ('a2', 'b2'): 15,
    ('a2', 'b3'): 16
}

# Создаем переменные решения
x = {(s, c): model.integer_var(name=f'x_{s}_{c}', lb=0)
      for s in suppliers for c in consumers}

# Добавляем ограничения по запасам
for s in suppliers:
    model.add_constraint(
        sum(x[(s, c)] for c in consumers) <= supply[s],
        ctname=f'supply_{s}'
    )

# Добавляем ограничения по спросу
for c in consumers:
    model.add_constraint(
        sum(x[(s, c)] for s in suppliers) >= demand[c],
        ctname=f'demand_{c}'
    )

# Определяем целевую функцию (минимизация стоимости)
total_cost = sum(x[(s, c)] * costs[(s, c)]
              for s in suppliers for c in consumers)
model.minimize(total_cost)

# Решаем задачу
solution = model.solve()

# Выводим результаты
if solution:
    print("Статус решения:", model.solve_details.status)
    print(f"Общая стоимость: {solution.get_objective_value():.0f}\n")

    print("Оптимальный план перевозок:")
    for s in suppliers:
        for c in consumers:
            val = solution.get_value(x[(s, c)])
            if val > 0:
                print(f"Из {s} в {c}: {val:.0f} ед. (стоимость: {costs[(s, c)]*val:.0f})")

    # Дополнительная информация о решении
    print("\nДополнительная информация:")
    print(f"Время решения: {model.solve_details.time:.2f} сек")
    print(f"Количество итераций: {model.solve_details.nb_iterations}")
else:
    print("Решение не найдено")

Статус решения: integer optimal solution
Общая стоимость: 383

Оптимальный план перевозок:
Из a1 в b1: 8 ед. (стоимость: 56)
Из a1 в b2: 2 ед. (стоимость: 18)
Из a2 в b2: 11 ед. (стоимость: 165)
Из a2 в b3: 9 ед. (стоимость: 144)

Дополнительная информация:
Время решения: 0.00 сек
Количество итераций: 0
